The aim is to pass the images that we have to tensors, to be able to work later with a convolutional neural network.

In [10]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torchmetrics
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Install torchmetrics
!pip install torchmetrics


In [12]:

# Function to get the dataset
def getData(path='archive/OriginalDataset', RNN=False):
    dataset = ImageFolder(root=path)
    transform1 = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        transforms.Grayscale(num_output_channels=1) if RNN else transforms.Lambda(lambda x: x)
    ])
    dataset = ImageFolder(root='archive/OriginalDataset', transform=transform1)
    return dataset



In [13]:
# Simple CNN model with 4 classes
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 16 * 16, 4),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



In [ ]:

class ImageRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ImageRNN, self).__init__()

        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        batch_size, channels, height, width = x.size()

        x = x.view(batch_size, channels, -1)

        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out




In [ ]:
class PLModel(pl.LightningModule):
    def __init__(self, model, num_classes):
        super(PLModel, self).__init__()
        self.model = model
        self.accuracy = torchmetrics.Accuracy('multiclass', num_classes=num_classes)
        self.f1 = torchmetrics.F1(average='macro', num_classes=num_classes)
        self.f1_each = torchmetrics.F1(num_classes=num_classes)
        self.loss = nn.functional.cross_entropy

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)

        accuracy = self.accuracy(y_hat, y)
        f1_macro = self.f1(y_hat, y)
        f1_each = self.f1_each(y_hat, y)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_accuracy', accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_f1_macro', f1_macro, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_f1_each', f1_each, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)

        accuracy = self.accuracy(y_hat, y)
        f1_macro = self.f1(y_hat, y)
        f1_each = self.f1_each(y_hat, y)

        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_accuracy', accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_f1_macro', f1_macro, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_f1_each', f1_each, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss



In [ ]:

class DataModule(pl.LightningDataModule):
    def __init__(self, Dataset, batch_size=32):
        super().__init__()
        self.Dataset = Dataset
        self.batch_size = batch_size

    def setup(self, stage=None):
        dataset = self.Dataset
        train_size = int(0.8 * len(dataset))
        val_size = int(0.1 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        self.train_dataset, self.val_dataset, self.test_dataset = torch.utils.data.random_split(
            dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)



In [ ]:

# Get the RNN dataset and train the model
dataset = getData(RNN=True)
dataset = DataModule(dataset, batch_size=32)
m = ImageRNN(16384, 128, 4)
model = PLModel(m, 4)

trainer = pl.Trainer(accelerator="gpu", max_epochs=10)
trainer.fit(model, dataset)

# Get the CNN dataset and train the model
dataset = getData()
dataset = DataModule(dataset, batch_size=32)
m = SimpleCNN()
model = PLModel(m, 4)

trainer = pl.Trainer(accelerator="gpu", max_epochs=10)
trainer.fit(model, dataset)

# Get the ResNet dataset and train the model
dataset = getData()
dataset = DataModule(dataset, batch_size=32)
m = Resnet()
model = PLModel(m, 4)

trainer = pl.Trainer(accelerator="gpu", max_epochs=10)
trainer.fit(model, dataset)
